In [17]:
import getpass
import os
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_groq import ChatGroq
from dotenv import load_dotenv

In [18]:
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
os.environ['TAVILY_API_KEY'] = os.getenv('TAVILY_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = os.getenv('LANGCHAIN_TRACING_V2')
os.environ['LANGCHAIN_ENDPOINT'] = os.getenv('LANGCHAIN_ENDPOINT')
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')

In [19]:
from langchain_core.tools import tool

In [20]:
@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiplica dos números enteros."""
    return first_int * second_int


@tool
def add(first_int: int, second_int: int) -> int:
    """Suma dos números enteros."""
    return first_int + second_int

@tool
def exponentiate(base: int, exponent: int) -> int:
    """Exponenciar la base a la potencia del exponente."""
    return base**exponent

In [21]:
tools = [add, exponentiate, multiply]

In [22]:
from langchain.tools.render import render_text_description

In [23]:
from langchain_core.prompts import ChatPromptTemplate

In [24]:
llm = ChatGroq(temperature=0, model="llama3-70b-8192")

In [25]:
from operator import itemgetter

In [26]:
def tool_chain(model_output):
    tool_map = {tool.name: tool for tool in tools}
    chosen_tool = tool_map[model_output["name"]]
    return itemgetter("arguments") | chosen_tool

In [27]:
rendered_tools = render_text_description(tools)

In [28]:
system_prompt = f"""Usted es un asistente que tiene acceso al siguiente conjunto de herramientas. Estos son los nombres y descripciones de cada herramienta:

{rendered_tools}

Dada la entrada del usuario, devuelva únicamente el nombre y los argumentos de la herramienta a utilizar en formato JSON válido. La respuesta debe ser un JSON con las claves 'name' y 'arguments', donde 'arguments' es un diccionario con los argumentos de la herramienta. No incluya ninguna otra explicación en la respuesta.
"""

In [29]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt), 
        ("user", "{input}")
    ]
)

In [30]:
from langchain_core.output_parsers import JsonOutputParser

In [35]:
chain = prompt | llm | JsonOutputParser() | tool_chain
chain

ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="Usted es un asistente que tiene acceso al siguiente conjunto de herramientas. Estos son los nombres y descripciones de cada herramienta:\n\nadd: add(first_int: int, second_int: int) -> int - Suma dos números enteros.\nexponentiate: exponentiate(base: int, exponent: int) -> int - Exponenciar la base a la potencia del exponente.\nmultiply: multiply(first_int: int, second_int: int) -> int - Multiplica dos números enteros.\n\nDada la entrada del usuario, devuelva únicamente el nombre y los argumentos de la herramienta a utilizar en formato JSON válido. La respuesta debe ser un JSON con las claves 'name' y 'arguments', donde 'arguments' es un diccionario con los argumentos de la herramienta. No incluya ninguna otra explicación en la respuesta.\n")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])
| ChatGroq

In [36]:
resultado = chain.invoke({"input": "¿Cuanto es tres por 1132?"})
resultado

3396

# Devolver entradas de herramientas

In [37]:
from langchain_core.runnables import RunnablePassthrough

In [38]:
chain = (
    prompt | llm | JsonOutputParser() | RunnablePassthrough.assign(output=tool_chain)
)
chain

ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="Usted es un asistente que tiene acceso al siguiente conjunto de herramientas. Estos son los nombres y descripciones de cada herramienta:\n\nadd: add(first_int: int, second_int: int) -> int - Suma dos números enteros.\nexponentiate: exponentiate(base: int, exponent: int) -> int - Exponenciar la base a la potencia del exponente.\nmultiply: multiply(first_int: int, second_int: int) -> int - Multiplica dos números enteros.\n\nDada la entrada del usuario, devuelva únicamente el nombre y los argumentos de la herramienta a utilizar en formato JSON válido. La respuesta debe ser un JSON con las claves 'name' y 'arguments', donde 'arguments' es un diccionario con los argumentos de la herramienta. No incluya ninguna otra explicación en la respuesta.\n")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])
| ChatGroq

In [39]:
resultado = chain.invoke({"input": "¿Cuanto es mil ciento veinte más 11098?"})
resultado

{'name': 'add',
 'arguments': {'first_int': 1120, 'second_int': 11098},
 'output': 12218}